<a href="https://colab.research.google.com/github/ofoofoo/MoEViT/blob/main/mlp_and_kan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [34]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define transforms for the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Initialize the model, loss function, and optimizer
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
validation_accuracies = []
for epoch in range(15):  # Number of epochs
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    model.train()
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation
    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    validation_accuracy = 100 * correct / total
    validation_accuracies.append(validation_accuracy)
    print(f"Epoch {epoch+1}, Validation Accuracy: {validation_accuracy:.2f}%")

print('Finished Training')

# Output the model and validation accuracies
print("Final model:")
print(model)
print("Validation accuracies for each epoch:")
print(validation_accuracies)

torch.save(model.state_dict(), 'cifar10_mlp_model_state_dict.pth')

Files already downloaded and verified
Files already downloaded and verified


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d6f890c3640>Exception ignored in: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7d6f890c3640>    self._shutdown_workers()
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__

  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
        if w.is_alive():self._shutdown_workers()

  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
        if w.is_alive():assert self._parent_pid == os.getpid(), 'can only test a child process'

AssertionError:   File "/usr/lib/python3.10/multiprocessing/pro

Epoch 1, Validation Accuracy: 45.34%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d6f890c3640>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d6f890c3640>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

Epoch 2, Validation Accuracy: 47.93%
Epoch 3, Validation Accuracy: 50.50%
Epoch 4, Validation Accuracy: 51.39%
Epoch 5, Validation Accuracy: 51.48%
Epoch 6, Validation Accuracy: 50.80%
Epoch 7, Validation Accuracy: 52.47%
Epoch 8, Validation Accuracy: 51.44%
Epoch 9, Validation Accuracy: 52.32%
Epoch 10, Validation Accuracy: 53.42%
Epoch 11, Validation Accuracy: 52.19%
Epoch 12, Validation Accuracy: 52.87%
Epoch 13, Validation Accuracy: 52.17%
Epoch 14, Validation Accuracy: 51.98%
Epoch 15, Validation Accuracy: 52.19%
Finished Training
Final model:
MLP(
  (fc1): Linear(in_features=3072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)
Validation accuracies for each epoch:
[45.34, 47.93, 50.5, 51.39, 51.48, 50.8, 52.47, 51.44, 52.32, 53.42, 52.19, 52.87, 52.17, 51.98, 52.19]


In [29]:
!git clone https://github.com/ZiyaoLi/fast-kan.git

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys

# Adjusting the path to include the directory containing the FastKAN module
sys.path.append('/content/fast-kan/fastkan')

# Importing FastKAN
from fastkan import FastKAN

# Define transforms for the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Instantiate the FastKAN model with appropriate arguments
model = FastKAN(
    layers_hidden=[3 * 32 * 32, 512, 256, 10],  # Input size is 3 * 32 * 32 for CIFAR-10, output size is 10
    grid_min=-2.0,
    grid_max=2.0,
    num_grids=8,
    use_base_update=True,
    base_activation=torch.nn.ReLU(),
    spline_weight_init_scale=0.1
)


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs.view(inputs.size(0), -1))  # Flatten the input data
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images.view(images.size(0), -1))  # Flatten the input data
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')


fatal: destination path 'fast-kan' already exists and is not an empty directory.
Files already downloaded and verified
Files already downloaded and verified


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


KeyboardInterrupt: 